In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
imdb_data = pd.read_csv("data/desafio_indicium_imdb.csv")

# Questão 1

Inicialmente, foi realizado uma limpeza e preparação da base de dados para garantir sua consistência e facilitar a modelagem futura.


Algumas variáveis, como Genre, foram segmentadas para extrair o gênero principal.
A variável Gross foi convertida em milhos, Runtime para número inteiro.




In [ ]:
imdb_data.head()

In [ ]:
imdb_data.info()

In [ ]:
imdb_data.describe()

In [ ]:
pd.DataFrame({
    'Nulos (total)': imdb_data.isnull().sum(),
    'Nulos (%)': (imdb_data.isnull().sum()/len(imdb_data)) * 100
})

In [ ]:
imdb_data.nunique()

In [ ]:
imdb_data.duplicated().sum()

In [ ]:
imdb_data.dtypes

Verificou-se as tipos de dados de cada coluna, identificou-se colunas numéricas armazenadas como texto.

In [ ]:
imdb_data['Released_Year'] = pd.to_numeric(imdb_data['Released_Year'], errors='coerce')
imdb_data['Runtime'] = imdb_data['Runtime'].astype(str).str.extract(r"(\d+)").astype(float)
imdb_data['Gross'] = imdb_data['Gross'].str.replace(',', '').astype(float)
imdb_data['Gross_million'] = imdb_data['Gross']/1_000_000

In [ ]:
imdb_data['Main_Genre']= imdb_data['Genre'].str.split(",").str[0]

Foram removidas duplicatas e registros irrelevantes. Após isso, seguiremos a etapa de análise exploratória com histograma e correlação.

In [ ]:
sns.histplot(imdb_data['IMDB_Rating'])
plt.title("Distribuição da Nota IMDB")
plt.xlabel("Nota IMDB")
plt.ylabel("Frequência")
plt.show()

In [ ]:
sns.histplot(imdb_data['Meta_score'].dropna())
plt.title("Distribuição do Meta Score")
plt.xlabel("Meta Score")
plt.ylabel("Frequência")
plt.show()

In [ ]:
sns.boxplot(data=imdb_data, x="Main_Genre", y="Gross_million")
plt.title("Distribuição do Faturamento por Gênero Principal")
plt.xlabel("Gênero")
plt.ylabel("Gross(USD)")
plt.xticks(rotation=30)
plt.show()

In [ ]:
sns.boxplot(data=imdb_data, x="Certificate", y="IMDB_Rating")
plt.title("Nota IMDB por Classificação Etária")
plt.xlabel("Classificação")
plt.ylabel("Nota IMDB")
plt.xticks(rotation=30)
plt.show()

In [ ]:
colunas_correlacao = ["IMDB_Rating", "Meta_score", "No_of_Votes", "Gross_million", "Runtime"]

correlacao = imdb_data[colunas_correlacao].corr()

sns.heatmap(correlacao, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlação entre variáveis numéricas")
plt.show()

# Questão 2

#### Questão 2a

Para recomendar um filme a uma pessoa desconhecida, consideraremos uma obra com qualidade reconhecida e apelo universal.
Utilizaremos a fórmula ponderada do [IMDB](https://help.imdb.com/article/imdb/track-movies-tv/weighted-average-ratings/GWT2DSBYVT2F25SK), conforme mostrado a seguir. Combina a nota do filme (`IMDB_Rating`) com o número de votos recebidos (`No_of_Vote`), reduzindo o viés de filmes com poucas avaliações.

$WR = \frac{v}{v+m} \times  R + \frac{m}{v+m} \times C$

$R =$ `IMDB_Rating`

$v =$ `No_of_Votes`

$C =$ média global das notas

$m =$ valor mínimo de votos

In [ ]:
C = imdb_data['IMDB_Rating'].mean()
m = imdb_data['No_of_Votes'].quantile(0.80)

imdb_data_populares = imdb_data[imdb_data['No_of_Votes'] > m].copy()

imdb_data_populares['weighted_rating'] = (
(imdb_data_populares['No_of_Votes'] / (imdb_data_populares['No_of_Votes'] +m)) * imdb_data_populares['IMDB_Rating'] +
                                         (m/ (imdb_data_populares['No_of_Votes'] + m)) * C
)

imdb_data_populares.sort_values('weighted_rating', ascending=False)[
    ['Series_Title', 'IMDB_Rating','No_of_Votes', 'weighted_rating']
].head(10)

Após aplicar a fórmula com `m` definido no percentil `80` dos votos, obtivemos um ranking dominado por títulos populares e bem avaliados.

#### Questão 2b

Para identificar os principais fatores associados ao faturamento de um filme (`Gross`), analisamos sua correlação com variáveis numéricas e também com atributos categóricos, como gênero e década de lançamento; essas últimas derivadas na etapa de EDA.

In [ ]:
colunas_correlacao = ["Gross_million", "No_of_Votes", "IMDB_Rating", "Meta_score", "Runtime"]
correlacao = imdb_data[colunas_correlacao].corr()
correlacao['Gross_million'].sort_values(ascending=False)

In [ ]:
imdb_data.groupby('Main_Genre')['Gross_million'].mean().sort_values(ascending=False)

In [ ]:
imdb_data.groupby('Certificate')['Gross_million'].mean().sort_values(ascending=False)

In [ ]:
imdb_data['Decade'] = (imdb_data['Released_Year'] // 10) * 10
imdb_data.groupby('Decade')['Gross_million'].mean().sort_values(ascending=False)

Entre as variáveis númericas, `No_of_Votes` tem a relação mais forte com `Gross`, o que é esperado, visto que filmes populares arrecadam mais.

Analisando por `Main_Genrer`, conseguimos observar que Family, Action e Animation costumam ter faturamentos médios mais altos.

Por década, os anos 2010 se destacam, isso sugere que os blockbusters modernos possuem muito alcance.

Todos os fatores citados são consequência do sucesso, não necessariamente a causa. Para uma análise preditiva antes do lançamento, um bom caminho de análise seria focar em variáveis como `Main_Genre`, `Director`, Stars, `Runtime` e até mesmo `Overview`.


#### Questão 2c

A coluna `Overview` contém descrições curtas dos filmes, e, representa um campo de texto livre que resume a narrativa ou o tema da obra.

Inicialmente, nessa análise, iremos pré-processar o texto, removendo pontuação, normalizando as letras e eliminando símbolos. A próxima etapa é selecionar os gêneros mais frequentes no conjunto de dados, e, para cada um deles, iremos gerar uma nuvem de palavras.

In [ ]:
from wordcloud import WordCloud

imdb_data['Overview_clean'] = imdb_data['Overview'].str.lower().str.replace(r'[^a-zA-Z ]', '', regex=True)

In [ ]:
from wordcloud import WordCloud

top_genres = imdb_data['Main_Genre'].value_counts().head(10).index.tolist()

for genre in top_genres:
  text = " ".join(imdb_data[imdb_data['Main_Genre'] == genre]['Overview_clean'].dropna())
  wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

  plt.figure(figsize=(10,5))
  plt.imshow(wordcloud, interpolation='bilinear')
  plt.axis('off')
  plt.title(f'Palavras mais comuns em {genre.upper()}')
  plt.show()

Essa abordagem trouxe padrões interessantes, observando a predominância de palavras de cada gênero mostrado:


* Drama: *love, life, woman, family* e *struggle*
    * sugere temas emocionais e relacionais
* Action: *must, fight, save, lead* e *battle*
    * sugere conflitos, urgência e ação direta
* Comedy: *friend, tries, fall, love, meet*
    * sugere situações sociais, relacionamentos
* Crime: *murder, police, detective, killer, investigate*
    * palavras diretamente relacionadas à narrativa policial e investigativa
* Biography: *life, story, become, career, history*
    * palavras voltadas à trajetória e transformação
* Animation: *world, young, boy, girl, adventure, friend*
    * sugere narrativas voltadas para público infantojunvenil com forte carga de imaginação e fantasia
* Horror: *haunted, mysterious, run, becomes, killer*
    * sugere medo, perseguição e suspense
* Mistery: *murderer, detective, missing, wife, apartment*
    * sugere tramas com tensão pscilogógica
* Western: *bounty, joins, hunting, gold, scam*
    * são termos associados ao velho oeste, a busca por riquezas, a lei do mais forte


Foi possível observar que, mesmo em descrições curtas, existe vocabulário suficiente para sugerir o gênero predominante do filme. É um comportamento evidente em gêneros com linguagem marcante. Podemos confirmar que essa coluna possui potencial para ser usada na inferência do gênero.

# Questão 3

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.impute import SimpleImputer

imdb_data_copy = imdb_data.copy()


features = ['Runtime', 'No_of_Votes', 'Gross_million', 'Main_Genre', 'Certificate', 'Decade']
target = 'IMDB_Rating'

df_model = imdb_data_copy[features + [target]].dropna()

X = df_model[features]
y = df_model[target]

colunas_numericas = ['Runtime','No_of_Votes', 'Gross_million']
colunas_categoricas = ['Main_Genre', 'Certificate', 'Decade']

In [ ]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

In [ ]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('numerico', numeric_transformer, colunas_numericas),
        ('categorico', categorical_transformer, colunas_categoricas)
    ]
)

In [ ]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
rmse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

In [ ]:
rmse

In [ ]:
r2

In [ ]:
mae

Explique como você faria a previsão da nota do imdb a partir dos dados.

Quais variáveis e/ou suas transformações você utilizou e por quê?

Qual tipo de problema estamos resolvendo (regressão, classificação)?

Qual modelo melhor se aproxima dos dados e quais seus prós e contras?

Qual medida de performance do modelo foi escolhida e por quê?

A partir dos dados existentes da base, para realizar a previsão da nota dos filmes, o problema será definido como regressão, visto que temos que prever uma variável numérica contínua.

A análise irá partir do princípio de utilizar informações disponíveis para entender o que influencia a avaliação crítica de um filme.

As variáveis utilizadas foram `Runtime`, `No_of_Votes`, `Gross_million`, `Main_Genre`, `Certificate`, `Decade`. Importante ressaltar, que, `Gross_million` originou-se a partir da conversão da receita bruta, `Main_Genre` foi uma separação dos gêneros que os filmes possuem, e, `Decade` é a extração do ano de lançamento do filme. A justificativa para a escolha é o potencial impacto na nota, além da disponibilidade nos dados e correlação como foi possível observar na Questão 1.

O tratamento dos dados envolveu limpeza, substituição de valores ausentes e codificação de categorias usando OneHotEncoding, incorporado a um ColumnTransformer para manter o pipeline organizado e escalável.

O modelo utilizado foi `Random Forest Regressor` por sua eficácia em dados tabulares, lidar bem com variáveis categóricas já codificadas e apresentar baixa sensibilidade a outliers. Os prós dessa escolha, destaca-se a capacidade de generalização e a interpretação através da importância das features. Como contras, o modelo pode demandar mais recursos computacionais e não ser tão interpretável quanto modelos lineares.

A avaliação foi feita com as métricas: `Root Mean Squared Error` (RMSE), `Mean Absolute Error` (MAE) e `R² Score`.

* RMSE penaliza erros grandes, o que é util para esse contexto em que pequenas variações na nota podem impactar a percepção do filme; o modelo trouxe resultado satisfatório abaixo de 0.6, e `R²` acima de 0.7.

Em conclusão, os resultados indicam que o modelo é capaz de capturar boa parte da variação da nota com as variáveis estabelecidas.

# Questão 4

In [ ]:
novo_filme = {
    'Series_Title': 'The Shawshank Redemption',
    'Released_Year': '1994',
    'Certificate': 'A',
    'Runtime': '142 min',
    'Genre': 'Drama',
    'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
    'Meta_score': 80.0,
    'Director': 'Frank Darabont',
    'Star1': 'Tim Robbins',
    'Star2': 'Morgan Freeman',
    'Star3': 'Bob Gunton',
    'Star4': 'William Sadler',
    'No_of_Votes': 2343110,
    'Gross': '28,341,469'
}

In [ ]:
df_novo_filme = pd.DataFrame([novo_filme])
df_novo_filme['Runtime'] = df_novo_filme['Runtime'].str.replace('min', '').astype(float)


df_novo_filme['Gross'] = df_novo_filme['Gross'].str.replace(',','')
df_novo_filme['Gross'] = pd.to_numeric(df_novo_filme['Gross'], errors='coerce')
df_novo_filme['Gross_million'] = df_novo_filme['Gross'] / 1_000_000


df_novo_filme['Main_Genre'] = df_novo_filme['Genre'].str.split(',').str[0]


df_novo_filme['Released_Year'] = pd.to_numeric(df_novo_filme['Released_Year'], errors='coerce')
df_novo_filme['Decade'] = (df_novo_filme['Released_Year'] // 10) * 10

In [ ]:
colunas_entrada = [
    'Main_Genre', 'Certificate', 'Decade', 'Runtime', 'Meta_score',
    'No_of_Votes', 'Gross_million'
]

In [ ]:
nota_prevista = model.predict(df_novo_filme[colunas_entrada])

print(f'Nota IMDB prevista: {nota_prevista[0]: .2f}')

# Questão 5

In [ ]:
import joblib

joblib.dump(model, 'modelo_imdb_alice.pkl')

# Questão 6

In [ ]:
pip freeze